In [1]:
import pandas as pd
import ast

In [2]:
# Load CSV with engineered features
df = pd.read_csv("datasets/security/train_preprocessed.csv")

In [3]:
# Convert string lists back to real lists
df["input_ids"] = df["input_ids"].apply(ast.literal_eval)
df["attention_mask"] = df["attention_mask"].apply(ast.literal_eval)
df["target"] = df["target"].astype(int)

In [4]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["target"], random_state=42)

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class CodeTokenDataset(Dataset):
    def __init__(self, df):
        self.inputs = df["input_ids"].tolist()
        self.masks = df["attention_mask"].tolist()
        self.labels = df["target"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.inputs[idx], dtype=torch.long),
            torch.tensor(self.masks[idx], dtype=torch.long),
            torch.tensor(self.labels[idx], dtype=torch.long)
        )

In [8]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    input_ids, attention_masks, labels = zip(*batch)

    input_ids = [x.clone().detach() if isinstance(x, torch.Tensor) else torch.tensor(x, dtype=torch.long) for x in input_ids]
    attention_masks = [x.clone().detach() if isinstance(x, torch.Tensor) else torch.tensor(x, dtype=torch.long) for x in attention_masks]

    labels = torch.tensor(labels, dtype=torch.long)

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return input_ids, attention_masks, labels

In [9]:
train_dataset = CodeTokenDataset(train_df)
val_dataset = CodeTokenDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn)

In [10]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids)
        packed_output, (hidden, _) = self.lstm(embedded)
        out = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)  # bidirectional
        return self.fc(self.dropout(out))

In [11]:
from sklearn.metrics import accuracy_score
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Detect vocab size from max token ID in dataset
vocab_size = max(df["input_ids"].explode()) + 1

# Initialize model
model = LSTMClassifier(vocab_size=vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [12]:
!pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from tqdm import tqdm

epochs = 15

for epoch in range(epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    for input_ids, attention_mask, labels in loop:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # Update progress bar
        avg_loss = train_loss / len(train_loader)
        loop.set_postfix(avg_loss=avg_loss, accuracy=100 * correct / total)


    # Validation after each epoch
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for input_ids, attention_mask, labels in val_loader:
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1).cpu().tolist()
            all_preds.extend(preds)
            all_labels.extend(labels.tolist())

    val_acc = accuracy_score(all_labels, all_preds)
    print(f" Epoch {epoch+1} complete - Train Accuracy: {100 * correct / total:.2f}% - Val Accuracy: {val_acc:.4f}")


Epoch 1/15:   1%|          | 6/547 [00:09<14:25,  1.60s/it, accuracy=43.8, avg_loss=0.00769]